In [1]:
import pandas as pd

resp = pd.read_csv('/Users/FrancescaSchott/sfdat/sfdat28-schott/project/data/ehresp_2014.csv')
atus = pd.read_csv('/Users/FrancescaSchott/sfdat/sfdat28-schott/project/data/atus_2014.csv')

In [2]:
##################################################################
# Choosing select features from ATUS Data to merge with EH Data #
##################################################################

atus_test = atus[['TUCASEID', 'TRERNWA', 'TRMJIND1', 'TRMJOCGR', 'TELFS', 'TREMODR', 'TRNUMHOU']]

atus_test.rename(columns={'TUCASEID':'tucaseid',
                         'TRERNWA':'income_weekly',
                         'TRMJIND1':'maj_ind',
                         'TRMJOCGR':'cat_occ',
                         'TELFS':'labor_status',
                         'TREMODR':'eh_resp',
                         'TRNUMHOU':'household_number'},
                 inplace=True)

/Users/FrancescaSchott/anaconda/envs/testenv/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [3]:
################################
# Merge ATUS Data with EH Data #
# New dataframe called 'resp'  #
################################

resp = pd.merge(resp, atus_test, on=['tucaseid'], how='left')

In [4]:
############################################
# Rename columns in resp for understanding #
############################################

resp.rename(columns={'erbmi':'bmi', 
                     'ertpreat':'primary_eating', 
                     'ertseat':'secondary_eating', 
                     'euexercise':'exercise', 
                     'eufastfd':'fast_food',
                     'eufdsit':'food_amount',
                     'euinclvl':'income_14_15',
                     'erincome':'income_lvl',
                     'eustores':'stores'
                     },
            inplace=True)                    

In [5]:
##################################
# Remove unused columns in resp #
#################################

resp.drop(['eeincome1', 
           'erhhch',
           'eudietsoda',
           'eudrink',
           'eueat',
           'euexfreq',
           'eufastfdfrq',
           'euincome2', 
           'erspemch', 
           'ethgt', 
           'etwgt', 
           'euffyday', 
           'eufinlwgt', 
           'eusnap', 
           'eugenhth',
           'eugroshp',
           'euhgt',
           'eumeat',
           'eumilk',
           'euprpmel',
           'eusoda',
           'eustreason',
           'eutherm',
           'euwgt',
           'euwic',
           'exincome1'
          ], axis=1, inplace=True)

In [6]:
#############################################################################################
# Check value counts for respondent's income level based on 2014 or 2015 poverty threshold #
#############################################################################################

resp.income_14_15.value_counts()

5    9232
6    1980
Name: income_14_15, dtype: int64

In [7]:
################################################
# Create subset of data based on:              #
#                                              #
# 2014 poverty threshold                       #
# household size of 1                          #
# employed (labor status)                      #
#                                              #
# And then drop those columns from data frame. #
################################################

resp = resp[resp['income_14_15'] == 5]
resp = resp[resp['household_number'] == 1]
resp = resp[resp['labor_status'] == 1]
resp = resp.drop(['tucaseid', 'tulineno', 'income_14_15', 'eh_resp', 'labor_status', 'household_number'], axis=1)

In [8]:
####################################################################
# Check response values for income level, exercise and food amount #
####################################################################

print resp.income_lvl.value_counts()
print resp.exercise.value_counts()
print resp.food_amount.value_counts()

 1    869
 5    136
 3     95
 2     50
-1      8
Name: income_lvl, dtype: int64
 1    775
 2    378
-1      3
-2      1
-3      1
Name: exercise, dtype: int64
 1    1089
 2      46
 3      17
-1       3
-2       2
-3       1
Name: food_amount, dtype: int64


In [9]:
####################################################################################
# Create binary values for income level                                            #
# Swicth exercise and food amount responses so 'good' is higher and 'bad' is lower #
####################################################################################

resp.income_lvl.replace([1, 2, 3, 5], [1, 0, 0, 0], inplace=True)
resp.exercise.replace([1, 2], [2, 1], inplace=True)
resp.food_amount.replace([1, 3], [3, 1], inplace=True)

In [10]:
##############################################################
# Drop rows without responses to income level - only 8 rows  #
##############################################################

resp = resp[resp.income_lvl != -1]

In [11]:
##########################################################################################
# Check current shape and new response values for income level, exercise and food amount #
##########################################################################################

print resp.shape
print resp.income_lvl.value_counts()
print resp.exercise.value_counts()
print resp.food_amount.value_counts()

(1150, 11)
1    869
0    281
Name: income_lvl, dtype: int64
 2    771
 1    377
-2      1
-3      1
Name: exercise, dtype: int64
 3    1084
 2      46
 1      17
-2       2
-3       1
Name: food_amount, dtype: int64


In [12]:
######################################################
# Make all negative values in data frame null values #
######################################################

resp.where(resp>=0, None, inplace=True)

resp.isnull().sum()

bmi                  56
income_lvl            0
primary_eating        0
secondary_eating      5
exercise              2
fast_food             3
food_amount           3
stores               26
income_weekly       131
maj_ind               0
cat_occ               0
dtype: int64

In [13]:
#############################################
# Fill null values based on feature         #
# Continuous features were filled with mean #
# Categorical features were filled with 0   #
# Weekly income decimal point was moved     #
#############################################

resp.bmi.fillna(resp.bmi.mean(), inplace=True)
resp.primary_eating = resp.primary_eating.apply(lambda x:float(x))
resp.secondary_eating.fillna(round(resp.secondary_eating.mean(), 0), inplace=True)

resp.exercise.fillna(0, inplace=True)
resp.fast_food.fillna(0, inplace=True)
resp.food_amount.fillna(0, inplace=True)
resp.stores.fillna(5, inplace=True)

resp.income_weekly.fillna(resp.income_weekly.mean(), inplace=True)
resp.income_weekly = resp.income_weekly.apply(lambda x: x/100)

resp.isnull().sum()

bmi                 0
income_lvl          0
primary_eating      0
secondary_eating    0
exercise            0
fast_food           0
food_amount         0
stores              0
income_weekly       0
maj_ind             0
cat_occ             0
dtype: int64

In [14]:
##################################################
# Reorder columns with response variables at end #
##################################################

resp = resp[['bmi', 'primary_eating', 'secondary_eating', 'exercise', 'fast_food', 'food_amount', 'stores', 'maj_ind', 'cat_occ', 'income_lvl', 'income_weekly']]

In [ ]:
###############################################################
# Save and export dataframe as csv to use in eda and modeling #
###############################################################

resp.to_csv('/Users/FrancescaSchott/sfdat/sfdat28-schott/project/data/resp.csv', index_col=0)